Importing Data & Libraries

In [4]:
# Import necessary libraries and modules

#Iheoma imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import io
plt.style.use('ggplot')

pd.options.mode.chained_assignment = None


# Import files into our working environment
df_2016 = pd.read_csv('properties_2016.csv', low_memory=False)
df_2017 = pd.read_csv('properties_2017.csv', low_memory=False)
zillow_data_dict = pd.read_excel('zillow_data_dictionary.xlsx')

# Replace column names with snake case for ease of manipulation
df_2016.columns=df_2016.columns.str.replace(" ","_").str.lower()
df_2017.columns=df_2017.columns.str.replace(" ","_").str.lower()
zillow_data_dict.columns=zillow_data_dict.columns.str.replace(" ","_").str.lower()

# Merge the two dataframes & drop duplicates
df = pd.concat([df_2016, df_2017], axis=0).drop_duplicates()

# Set the display options to show full text wrapping
pd.set_option('display.max_colwidth', None)

# display(zillow_data_dict)


Data Cleaning
*

In [5]:
# Determining list of variables to drop

# Dropping'taxdelinquencyflag' 'taxdelinquencyyear' as they don't impact our model
# Dropping other location data not needed 'regionidneighborhood' 'regionidcity' 'regionidcounty'
# -> will keep zipcode but need to remove data that is missing this info
# 'rawcensustractandblock','censustractandblock','latitude','longitude'
# dropping all extra zoning data 'propertyzoningdesc','propertycountylandusecode'
# dropping other tax columns'landtaxvaluedollarcnt','structuretaxvaluedollarcnt', 'taxamount'
# redunant size categories,'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50','finishedsquarefeet6'
# redundant bathroom references 'threequarterbathnbr','fullbathcnt', 'calculatedbathnbr'
# objects/flag/qualitative, 'hashottuborspa', 'buildingqualitytypeid', 'buildingclasstypeid', 'fireplaceflag','architecturalstyletypeid'
# pool size redundancy 'pooltypeid10','pooltypeid2','pooltypeid7','poolsizesum'
# Drop finishedfloor1squarefeet as column since 93% of data is missing this and is possibly duplicative with other values
drop_list = ['taxdelinquencyflag','taxdelinquencyyear','taxamount','landtaxvaluedollarcnt','structuretaxvaluedollarcnt','yardbuildingsqft26','typeconstructiontypeid','storytypeid','regionidneighborhood','regionidcity','regionidcounty','censustractandblock','rawcensustractandblock','propertyzoningdesc','propertylandusetypeid','propertycountylandusecode','pooltypeid7','pooltypeid2','pooltypeid10','longitude','latitude','heatingorsystemtypeid','hashottuborspa','fullbathcnt','fireplaceflag','finishedsquarefeet50','finishedsquarefeet15','finishedsquarefeet13','finishedsquarefeet12','finishedsquarefeet6','threequarterbathnbr','decktypeid','buildingclasstypeid','buildingqualitytypeid','architecturalstyletypeid','airconditioningtypeid','finishedfloor1squarefeet','calculatedbathnbr']
df.drop(drop_list, axis=1, inplace=True)

#Handling Missing Data
# Dropping rows with null for'parcelid','bathroomcnt','bedroomcnt','fips','taxvaluedollarcnt','assessmentyear'
df.dropna(subset=['parcelid', 'bathroomcnt', 'bedroomcnt', 'fips', 'taxvaluedollarcnt', 'assessmentyear', 'regionidzip', 'yearbuilt','roomcnt'], inplace=True)

missing_zero_columns = ['basementsqft','fireplacecnt','garagecarcnt','garagetotalsqft','poolcnt','poolsizesum','yardbuildingsqft17']
df[missing_zero_columns] = df[missing_zero_columns].fillna(0)

missing_one_columns = ['unitcnt','numberofstories']
df[missing_one_columns] = df[missing_one_columns].fillna(1)

# If all measures of house size are missing drop that data
df = df.dropna(subset=['calculatedfinishedsquarefeet', 'lotsizesquarefeet'], how='all')

# starting with calculated finishedsquarefeet(most important) if that data is missing fill with lotsize
df['calculatedfinishedsquarefeet'] = df['calculatedfinishedsquarefeet'].fillna(df['lotsizesquarefeet'])

# If lotsizemissing use calculatedfinished
df['lotsizesquarefeet'] = df['lotsizesquarefeet'].fillna(df['calculatedfinishedsquarefeet'])
def save_cleaned_dataframe(df, file_name):
    # Save the cleaned DataFrame to a CSV file
    df.to_csv(file_name, index=False)
    print(f"DataFrame saved to {file_name}")

save_cleaned_dataframe(df,'iheoma_cleaned.csv')


DataFrame saved to iheoma_cleaned.csv
